In [1]:
from data_utils.load import load_data, beautiful_head
import pandas_profiling
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
def reload_data():
    train = load_data('data/train2016.csv')
    test = load_data('data/test2016.csv')
    y_train = train.Party
    enc_label = LabelEncoder()
    y_train = enc_label.fit_transform(y_train)
    train.drop(['Party', 'USER_ID'], axis = 1, inplace = True)
    test.drop(['USER_ID'], axis = 1, inplace = True)
    
    return train, test, y_train

In [3]:
train = load_data('data/train2016.csv')
test = load_data('data/test2016.csv')
beautiful_head(train, size_chunk_columns=8)

,USER_ID,YOB,Gender,Income,HouseholdStatus,EducationLevel,Party,Q124742
0,1,1938,Male,MissingCategory,Married (w/kids),MissingCategory,Democrat,No
1,4,1970,Female,"over $150,000",Domestic Partners (w/kids),Bachelor's Degree,Democrat,MissingCategory
2,5,1997,Male,"$75,000 - $100,000",Single (no kids),High School Diploma,Republican,MissingCategory
3,8,1983,Male,"$100,001 - $150,000",Married (w/kids),Bachelor's Degree,Democrat,No
4,9,1984,Female,"$50,000 - $74,999",Married (w/kids),High School Diploma,Republican,No


,Q124122,Q123464,Q123621,Q122769,Q122770,Q122771,Q122120,Q121699
0,MissingCategory,No,No,No,Yes,Public,No,Yes
1,Yes,No,No,No,Yes,Public,No,Yes
2,Yes,Yes,No,MissingCategory,Yes,Private,No,No
3,Yes,No,Yes,No,No,Public,No,Yes
4,Yes,No,No,No,Yes,Public,No,Yes


,Q121700,Q120978,Q121011,Q120379,Q120650,Q120472,Q120194,Q120012
0,No,MissingCategory,No,No,Yes,MissingCategory,Try first,No
1,No,Yes,No,No,Yes,Science,Study first,Yes
2,No,Yes,No,No,Yes,Science,Study first,MissingCategory
3,No,Yes,No,No,Yes,Science,Try first,No
4,No,Yes,Yes,No,Yes,Art,Try first,Yes


,Q120014,Q119334,Q119851,Q119650,Q118892,Q118117,Q118232,Q118233
0,No,MissingCategory,Yes,MissingCategory,Yes,Yes,Idealist,No
1,Yes,No,No,Receiving,No,No,Pragmatist,No
2,Yes,No,Yes,Receiving,No,Yes,Pragmatist,No
3,Yes,Yes,No,Giving,Yes,No,Idealist,No
4,No,No,No,Giving,No,No,Idealist,Yes


,Q118237,Q117186,Q117193,Q116797,Q116881,Q116953,Q116601,Q116441
0,No,MissingCategory,MissingCategory,Yes,Happy,Yes,Yes,No
1,No,Cool headed,Standard hours,No,Happy,Yes,Yes,Yes
2,Yes,Cool headed,Odd hours,No,Right,Yes,No,No
3,No,Cool headed,Standard hours,No,Happy,Yes,Yes,No
4,Yes,Hot headed,Standard hours,No,Happy,Yes,Yes,No


...

,Q108342,Q108343,Q107869,Q107491,Q106993,Q106997,Q106272,Q106388
0,In-person,MissingCategory,Yes,No,Yes,Yay people!,Yes,No
1,In-person,No,Yes,Yes,No,Yay people!,Yes,Yes
2,In-person,No,No,Yes,Yes,Grrr people,Yes,No
3,Online,No,Yes,No,Yes,Grrr people,No,No
4,Online,No,No,Yes,Yes,Yay people!,Yes,No


,Q106389,Q106042,Q105840,Q105655,Q104996,Q103293,Q102906,Q102674
0,Yes,Yes,MissingCategory,No,Yes,No,No,No
1,Yes,Yes,Yes,No,Yes,MissingCategory,MissingCategory,MissingCategory
2,No,No,No,No,No,Yes,No,No
3,Yes,Yes,No,Yes,Yes,No,No,No
4,Yes,Yes,Yes,Yes,No,No,No,Yes


,Q102687,Q102289,Q102089,Q101162,Q101163,Q101596,Q100689,Q100680
0,Yes,No,Own,Optimist,MissingCategory,Yes,Yes,No
1,MissingCategory,MissingCategory,MissingCategory,MissingCategory,MissingCategory,MissingCategory,MissingCategory,MissingCategory
2,Yes,No,Own,Pessimist,Mom,No,No,No
3,Yes,Yes,Own,Optimist,Mom,No,No,No
4,No,No,Own,Optimist,Mom,No,Yes,Yes


,Q100562,Q99982,Q100010,Q99716,Q99581,Q99480,Q98869,Q98578
0,No,Nope,Yes,No,No,MissingCategory,No,MissingCategory
1,MissingCategory,MissingCategory,MissingCategory,MissingCategory,MissingCategory,No,No,No
2,No,Nope,Yes,No,No,No,Yes,No
3,Yes,Check!,No,No,No,Yes,Yes,No
4,Yes,Nope,Yes,No,No,Yes,No,No


,Q98059,Q98078,Q98197,Q96024
0,Only-child,No,No,Yes
1,Only-child,Yes,No,No
2,Yes,No,Yes,No
3,Yes,No,No,Yes
4,Yes,No,No,Yes


In [4]:
profile = pandas_profiling.ProfileReport(train)
profile

Number of variables,108
Number of observations,5568
Total Missing (%),0.0%
Total size in memory,4.6 MiB
Average record size in memory,864.0 B
Numeric,2
Categorical,106
Date,0
Text (Unique),0
Rejected,0
Distinct count,8


In [5]:
y_train = train.Party
y = train.Party
enc_label = LabelEncoder()
y_train = enc_label.fit_transform(y_train)
train.drop(['Party', 'USER_ID'], axis = 1, inplace = True)
test.drop(['USER_ID'], axis = 1, inplace = True)

# One Hot Encoding

In [6]:
def onehotencoding(train, test, features='all'):
    
    #import necessary modules
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    import numpy as np
    import pandas as pd
    
    #write only categorical names to variable cat_columns
    cat_columns_train = sorted([i for i in train.columns if train[i].dtype == 'O'])
    cat_columns_test = sorted([i for i in test.columns if test[i].dtype == 'O'])
    
    if cat_columns_train != cat_columns_test:
        raise ValueError('Train and test sets must have the same columns')
    
    if features == 'all':
        features = cat_columns_train 
    else:
        if type(features) != list:
            raise ValueError(str(type(features)) + ' type passed in features. Only list are allowed.')
        if len(features) == 0:
            raise ValueError('You have chosen not enough features. The minimum number is one.')
            
        feat_not_in_data = [i for i in features if i not in cat_columns_train]
        
        if len(feat_not_in_data) > 0:
            raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
            
    #save all labels
    labels = []
    
    # encode the first column
    enc_label = LabelEncoder()
    new_train = enc_label.fit_transform(train[features[0]])
    labels.append(enc_label.classes_)
    new_test = enc_label.transform(test[features[0]])
    
    # do the others
    for i in features[1:]:
        enc_label = LabelEncoder()
        new_train = np.column_stack((new_train, enc_label.fit_transform(train[i])))
        labels.append(enc_label.classes_)
        new_test = np.column_stack((new_test, enc_label.transform(test[i])))
    
    #do OneHotEncoding
    enc_onehot = OneHotEncoder()
    new_train = enc_onehot.fit_transform(new_train)
    new_test = enc_onehot.fit_transform(new_test)
    
    # create a list of columns to help create a DF from np array 
    new_cols = [features[i] + '_' + str(j) for i in range(0,len(features)) for j in labels[i] ]
    
    # create new dataframe
    new_train = pd.DataFrame(new_train.toarray(),columns=new_cols)
    new_test = pd.DataFrame(new_test.toarray(),columns=new_cols)
    for i in train.columns:
        if i not in features:
            new_train[i] = train[i]
            new_test[i] = test[i]
            
    return new_train, new_test

In [7]:
new_train, new_test = onehotencoding(train,test)

# Binary coding

In [8]:
def binarycoding(train, test, features='all'):
    #import necessary modules
    from sklearn.preprocessing import LabelEncoder
    import numpy as np
    import pandas as pd
    
    #write only categorical names to variable cat_columns
    cat_columns_train = sorted([i for i in train.columns if train[i].dtype == 'O'])
    cat_columns_test = sorted([i for i in test.columns if test[i].dtype == 'O'])
    
    if cat_columns_train != cat_columns_test:
        raise ValueError('Train and test sets must have the same columns')
    
    if features == 'all':
        features = cat_columns_train 
    else:
        if type(features) != list:
            raise ValueError(str(type(features)) + ' type passed in features. Only list are allowed.')
        if len(features) == 0:
            raise ValueError('You have chosen not enough features. The minimum number is one.')
            
        feat_not_in_data = [i for i in features if i not in cat_columns_train]
        
        if len(feat_not_in_data) > 0:
            raise ValueError(','.join(feat_not_in_data) + ' are not in data.')

    #save number of new columns
    labels = []
    l = 0
    
    #do binary coding
    def binary(x):
        max_x = len(bin(max(x))[2:])
        new_x = [list((max_x - len(bin(i)[2:]))*'0' + str(bin(i))[2:]) for i in x]
        return np.array(new_x, dtype = int), max_x
    
    # encode the first column
    enc_label = LabelEncoder()
    new_train, l = binary(enc_label.fit_transform(train[features[0]]))
    labels.append(l)
    new_test,_ = binary(enc_label.transform(test[features[0]]))
    
    # do the others
    for i in features[1:]:
        enc_label = LabelEncoder()
        col, l = binary(enc_label.fit_transform(train[i]))
        new_train = np.column_stack((new_train, col))
        labels.append(l)
        col, l = binary(enc_label.transform(test[i]))
        new_test = np.column_stack((new_test, col))
        
    # create a list of columns to help create a DF from np array 
    new_cols = [features[i] + '_' + str(j) for i in range(0,len(features)) for j in range(labels[i]) ]

    # create new dataframe
    new_train = pd.DataFrame(new_train,columns=new_cols)
    new_test = pd.DataFrame(new_test,columns=new_cols)
    for i in train.columns:
        if i not in features:
            new_train[i] = train[i]
            new_test[i] = test[i]    
    
    return new_train, new_test

In [9]:
new_train, new_test = binarycoding(train, test)

# Information Value and Weight of Evidence

Information Value | Predictive Power
------------- | -------------|
< 0.02	| useless for prediction
0.02 to 0.1	| Weak predictor
0.1 to 0.3	| Medium predictor
0.3 to 0.5	| Strong predictor
>0.5	| Suspicious or too good to be true

In [10]:
def woe(train, test, y, features='all', information_value=False, inplace=False):
    import numpy as np
    import pandas as pd
    if len(set(y)) != 2:
        raise ValueError('Transformation is only used for binary classification.')
    
    #write only categorical names to variable cat_columns
    cat_columns_train = sorted([i for i in train.columns if train[i].dtype == 'O'])
    cat_columns_test = sorted([i for i in test.columns if test[i].dtype == 'O'])
    
    if cat_columns_train != cat_columns_test:
        raise ValueError('Train and test sets must have the same columns')
    
    if features == 'all':
        features = cat_columns_train 
    else:
        if type(features) != list:
            raise ValueError(str(type(features)) + ' type passed in features. Only list are allowed.')
        if len(features) == 0:
            raise ValueError('You have chosen not enough features. The minimum number is one.')
            
        feat_not_in_data = [i for i in features if i not in cat_columns_train]
        
        if len(feat_not_in_data) > 0:
            raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
    
    
    # calculate weights of evidance
    def cal_woe(x_train, x_test, name):
        good = np.sum(y == 0)
        bad = np.sum(y == 1)
        inf_v = 0
        
        for i in x_train.unique():
            count_bad = float(np.sum((y == 1) & (x_train == i))) / bad
            count_good = float(np.sum((y == 0) & (x_train == i))) / good
            x_train[x_train==i] = np.log(count_good / count_bad)
            x_test[x_test==i] = np.log(count_good / count_bad)
            inf_v += (count_good - count_bad) * np.log(count_good / count_bad)
        
        return x_train, x_test, inf_v
    
    
    inf_value = dict()
    if inplace:
        
        for i in features:
            train[i], test[i], inf_value[i] = cal_woe(train[i], test[i], i)
        
        if information_value:
            return pd.DataFrame(inf_value.items(), 
                                      columns = ['Feature', 'Inf_value']).sort('Inf_value', ascending=False)
        
    else:
        
        new_train = train.copy()
        new_test = test.copy()
        for i in features:
            new_train[i], new_test[i], inf_value[i] = cal_woe(new_train[i], new_test[i], i)
        
        if information_value:
            return new_train, new_test, pd.DataFrame(inf_value.items(), 
                                      columns = ['Feature', 'Inf_value']).sort('Inf_value', ascending=False)
        return new_train, new_test

In [11]:
new_train, new_test, inf_value = woe(train, test, y_train, information_value=True)

/Users/vitaliyradchenko/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/vitaliyradchenko/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/vitaliyradchenko/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:63: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [12]:
inf_value.head()

,Feature,Inf_value
31,Q109244,0.315566
68,Q115611,0.113908
83,Q98197,0.084871
93,Q113181,0.081066
36,Gender,0.049024


# Combination of factors 

In [13]:
def create_combinations(data, number_of_interactions=2, features='all', inplace=False):
    
    from itertools import combinations
    
    #write only categorical names to variable cat_columns
    cat_columns = [i for i in data.columns if data[i].dtype == 'O'] 
    
    if features == 'all':
        features = [i for i in data.columns if data[i].dtype == 'O'] 
    else:
        if type(features) != list:
            raise ValueError(str(type(features)) + ' type passed in features. Only list are allowed.')
        if len(features) < 2:
            raise ValueError('You have chosen not enough features. The minimum number is two.')
            
        feat_not_in_data = [i for i in features if i not in cat_columns]
        
        if len(feat_not_in_data) > 0:
            raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
    
    temp = 0
    if not inplace:
        new_data = data.copy()
        
    for i in combinations(features, number_of_interactions):
        if len(i) > 2:
            temp = data[i[0]].astype(str).str.cat(data[i[1]].astype(str), sep='_')
            for j in i[2:]:
                temp = temp.astype(str).str.cat(data[j].astype(str), sep='_')
            
            name = '_'.join(i)
            if inplace:
                data[name] = temp
            else:
                new_data[name] = temp
        else:
            name = '_'.join(i)
            if inplace:
                data[name] = data[i[0]].astype(str).str.cat(data[i[1]].astype(str), sep='_')
            else:
                new_data[name] = data[i[0]].astype(str).str.cat(data[i[1]].astype(str), sep='_')
                
    if not inplace:
        return new_data

In [14]:
new_train = create_combinations(train, 3, features = ['Gender', 'Q109244', 'Q115611', 'Q98197', 'Q113181'], inplace = False)

In [15]:
new_train.head()

,YOB,Gender,Income,HouseholdStatus,EducationLevel,Q124742,Q124122,Q123464,Q123621,Q122769,...,Gender_Q109244_Q115611,Gender_Q109244_Q98197,Gender_Q109244_Q113181,Gender_Q115611_Q98197,Gender_Q115611_Q113181,Gender_Q98197_Q113181,Q109244_Q115611_Q98197,Q109244_Q115611_Q113181,Q109244_Q98197_Q113181,Q115611_Q98197_Q113181
0,1938,Male,MissingCategory,Married (w/kids),MissingCategory,No,MissingCategory,No,No,No,...,Male_No_No,Male_No_No,Male_No_No,Male_No_No,Male_No_No,Male_No_No,No_No_No,No_No_No,No_No_No,No_No_No
1,1970,Female,"over $150,000",Domestic Partners (w/kids),Bachelor's Degree,MissingCategory,Yes,No,No,No,...,Female_No_No,Female_No_No,Female_No_MissingCategory,Female_No_No,Female_No_MissingCategory,Female_No_MissingCategory,No_No_No,No_No_MissingCategory,No_No_MissingCategory,No_No_MissingCategory
2,1997,Male,"$75,000 - $100,000",Single (no kids),High School Diploma,MissingCategory,Yes,Yes,No,MissingCategory,...,Male_No_Yes,Male_No_Yes,Male_No_Yes,Male_Yes_Yes,Male_Yes_Yes,Male_Yes_Yes,No_Yes_Yes,No_Yes_Yes,No_Yes_Yes,Yes_Yes_Yes
3,1983,Male,"$100,001 - $150,000",Married (w/kids),Bachelor's Degree,No,Yes,No,Yes,No,...,Male_No_No,Male_No_No,Male_No_No,Male_No_No,Male_No_No,Male_No_No,No_No_No,No_No_No,No_No_No,No_No_No
4,1984,Female,"$50,000 - $74,999",Married (w/kids),High School Diploma,No,Yes,No,No,No,...,Female_No_No,Female_No_No,Female_No_No,Female_No_No,Female_No_No,Female_No_No,No_No_No,No_No_No,No_No_No,No_No_No


# Count Featurizer

In [23]:
class CountFeaturizer(object):

    def __init__(self, input_col, label_col, output_col='count_features', min_count_per_cat=2):
        self.input_col = input_col
        self.label_col = label_col
        self.output_col = output_col
        self.min_count_per_cat = min_count_per_cat

    def fit(self, data):
        import pandas as pd
        from copy import deepcopy
        
        self.train_set = data
        self.inputs = sorted(data[self.input_col].unique())
        self.labels = sorted(data[self.label_col].unique(), key=lambda x: '{}'.format(x))
        label_counts_tmp = data.groupby([self.label_col]).size()
        label_counts = pd.Series([label_counts_tmp.loc[i] for i in self.labels])
        pair_counts = data.groupby([self.input_col, self.label_col]).size()
        self.input_counts = data.groupby([self.input_col]).size()
        self.logodds = {}
        self.logodds_class = {}
        self.default_logodds = np.log(label_counts / (float(len(data)) - label_counts))
        for inp in self.inputs:
            self.logodds_class[inp] = np.log(self.input_counts[inp] / (float(len(data)) - self.input_counts[inp]))
            self.logodds[inp] = deepcopy(self.default_logodds)
            for cat in pair_counts[inp].keys():
                if self.min_count_per_cat < pair_counts[inp].loc[cat] < self.input_counts[inp]:
                    prob_inp = pair_counts[inp].loc[cat] / float(self.input_counts[inp])
                    self.logodds[inp][self.labels.index(cat)] = np.log(prob_inp) - np.log(1.0 - prob_inp)
            self.logodds[inp] = pd.Series(self.logodds[inp])
            self.logodds[inp].index = range(len(self.labels))

    def transform(self, data):
        import pandas as pd
        from copy import deepcopy
        
        new_inputs = sorted(data[self.input_col].unique())
        new_input_counts = data.groupby(self.input_col).size()
        only_new = set(new_inputs + self.inputs) - set(self.inputs)
        in_both = set(new_inputs).intersection(self.inputs)
        new_cnt = 0.0 if data is self.train_set else float(len(data))

        for inp in only_new:
            self.logodds_class[inp] = np.log(new_input_counts[inp] /
                                             (float(len(self.train_set) + new_cnt) - new_input_counts[inp]))
            self.logodds[inp] = deepcopy(self.default_logodds)
            self.logodds[inp].index = range(len(self.labels))
        if data is not self.train_set:
            for inp in in_both:
                prob_inp = (self.input_counts[inp] + new_input_counts[inp]) / float(len(self.train_set) + new_cnt)
                self.logodds_class[inp] = np.log(prob_inp) - np.log(1. - prob_inp)

        for i in range(0, len(self.labels)):
            data['{}_{}'.format(self.output_col, i)] = data[self.input_col].apply(lambda _: self.logodds[_].loc[i])
        data['{}_class'.format(self.output_col)] = data[self.input_col].apply(lambda _: self.logodds_class[_])
        return data

In [24]:
def count_featurizer(train, test, y, features='all'):
    
    #write only categorical names to variable cat_columns
    cat_columns_train = sorted([i for i in train.columns if (train[i].dtype == 'O') and (i != y)])
    cat_columns_test = sorted([i for i in test.columns if test[i].dtype == 'O'])
    
    if cat_columns_train != cat_columns_test:
        raise ValueError('Train and test sets must have the same columns')
    
    if type(y) == 'str':
        raise ValueError('Only string type are allowed')
    
    if features == 'all':
        features = cat_columns_train 
    else:
        if type(features) != list:
            raise ValueError(str(type(features)) + ' type passed in features. Only list are allowed.')
        if len(features) == 0:
            raise ValueError('You have chosen not enough features. The minimum number is one.')
            
        feat_not_in_data = [i for i in features if i not in cat_columns_train]
        
        if len(feat_not_in_data) > 0:
            raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
    
    new_train = train.copy()
    new_test = test.copy()
    
    for i in features:
        name_out = 'count_' + i
        cf = CountFeaturizer(i, y, output_col=name_out)
        cf.fit(train)
        new_train = cf.transform(new_train)
        new_test = cf.transform(new_test)
        
    return new_train, new_test

In [25]:
train = load_data('data/train2016.csv')
test = load_data('data/test2016.csv')
new_train, new_test = count_featurizer(train, test, 'Party', features=['Gender', 'Q109244', 'Q115611', 'Q98197', 'Q113181'])

# Mean Target

In [26]:
class MeanTarget(object):
    
    def checker(self):
        
        if self.features == 'all':
            self.features = self.cat_columns 
            
        else:
            if type(self.features) != list:
                raise ValueError(str(type(self.features)) + ' type passed in features. Only list are allowed.')
            if len(self.features) == 0:
                raise ValueError('You have chosen not enough features. The minimum number is one.')

            feat_not_in_data = [i for i in self.features if i not in self.cat_columns]

            if len(feat_not_in_data) > 0:
                raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
                
        if set(self.y) == 2:
            raise ValueError('Target variable is constant. It should be binary.')
        
    def fit(self, data, y, features='all'):
        import pandas as pd
        
        self.y = y
        self.cat_columns = sorted([i for i in data.columns if data[i].dtype == 'O'])
        self.features = features
        
        self.checker()
        
        unique_y = sorted(list(set(self.y)))
        self.values = dict()
        for col in self.features:
            self.values[col] = dict()
            temp = pd.DataFrame({'y' : y_train, col : train[col]}).groupby(['y', col]).size()
    
            for i, item in enumerate(temp[1].index):
                self.values[col][item] = float(temp[1][i]) / np.sum(temp[1])
                
    def fit_transform(self, data, y, features='all'):
        
        self.fit(data, y, features)
        
        for col in self.values:
            for item, value in self.values[col].iteritems():
                data.ix[data[col] == item, col] = value
    
    def transform(self, data):
        import warnings

        for col in self.values:
            if col in data.columns:
                for item, value in self.values[col].iteritems():
                    data.ix[data[col] == item, col] = value
            else:
                warnings.warn('Column ' + col + ' is missed in this dataset.')
    

In [27]:
train, test, y = reload_data()

In [28]:
mt = MeanTarget()
mt.fit(train, y_train, features=['Gender'])
mt.transform(test)

In [29]:
test.Gender.unique()

array([0.32670997325181506, 0.6541841803591899, 0.019105846388995033], dtype=object)

# Weight Mean Target

In [30]:
class WeightMeanTarget(object):
    
    def __init__(self, m=100):
        self.m = m
    
    def checker(self):
        
        if self.features == 'all':
            self.features = self.cat_columns 
            
        else:
            if type(self.features) != list:
                raise ValueError(str(type(self.features)) + ' type passed in features. Only list are allowed.')
            if len(self.features) == 0:
                raise ValueError('You have chosen not enough features. The minimum number is one.')

            feat_not_in_data = [i for i in self.features if i not in self.cat_columns]

            if len(feat_not_in_data) > 0:
                raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
                
        if set(self.y) == 2:
            raise ValueError('Target variable is constant. It should be binary.')
        
    def fit(self, data, y, features='all'):
        import pandas as pd
        import numpy as np
        
        self.y = y
        self.n = len(self.y)
        self.share = np.mean(y)
        self.cat_columns = sorted([i for i in data.columns if data[i].dtype == 'O'])
        self.features = features
        
        
        self.checker()
        
        unique_y = sorted(list(set(self.y)))
        self.values = dict()
        for col in self.features:
            self.values[col] = dict()
            temp = pd.DataFrame({'y' : y_train, col : train[col]}).groupby(['y', col]).size()
    
            for i, item in enumerate(temp[1].index):
                self.values[col][item] = self.prob_function(temp, i)
            
    def fit_transform(self, data, y, features='all'):
        
        self.fit(data, y, features)
        
        for col in self.values:
            for item, value in self.values[col].iteritems():
                data.ix[data[col] == item, col] = value
    
    def transform(self, data):
        import warnings

        for col in self.values:
            if col in data.columns:
                for item, value in self.values[col].iteritems():
                    data.ix[data[col] == item, col] = value
            else:
                warnings.warn('Column ' + col + ' is missed in this dataset.')
                
    def lambda_function(self, n):
        return float(n) / (self.m + n)
    
    def prob_function(self, temp, i):
        return float(temp[1][i]) / np.sum(temp[1]) * self.lambda_function(np.sum(temp[1])) + \
                  (1-self.lambda_function(np.sum(temp[1])))*self.share  

In [31]:
train, test, y = reload_data()

In [32]:
mt = WeightMeanTarget(500)
mt.fit(train, y, features=['Gender'])
mt.transform(test)

In [33]:
test.Gender.unique()

array([0.34969637213796056, 0.624640228410017, 0.09143522359769748], dtype=object)

# Stas Semenov approach

In [34]:
class SemenovEncoding(object):
    
    def __init__(self, C=10):
        self.C = C
    
    def checker(self):
        
        if self.features == 'all':
            self.features = self.cat_columns 
            
        else:
            if type(self.features) != list:
                raise ValueError(str(type(self.features)) + ' type passed in features. Only list are allowed.')
            if len(self.features) == 0:
                raise ValueError('You have chosen not enough features. The minimum number is one.')

            feat_not_in_data = [i for i in self.features if i not in self.cat_columns]

            if len(feat_not_in_data) > 0:
                raise ValueError(','.join(feat_not_in_data) + ' are not in data.')
                
        if set(self.y) == 2:
            raise ValueError('Target variable is constant. It should be binary.')
    
    def main_function(self, temp, i):
        return (temp[1][i] * (temp[0][i] + temp[1][i])/np.sum(temp) + 
                self.global_mean * self.C) / (temp[0][i] + temp[1][i] + self.C)
    
    def fit(self, data, y, features='all'):
        import pandas as pd
        import numpy as np
        
        self.y = y
        self.cat_columns = sorted([i for i in data.columns if data[i].dtype == 'O'])
        self.features = features
        self.global_mean = np.mean(y)
        
        self.checker()
        
        unique_y = sorted(list(set(self.y)))
        self.values = dict()
        for col in self.features:
            self.values[col] = dict()
            temp = pd.DataFrame({'y' : y_train, col : train[col]}).groupby(['y', col]).size()
        
            for i, item in enumerate(temp[1].index):
                self.values[col][item] = self.main_function(temp, i)
                
        return self.values
            
    def fit_transform(self, data, y, features='all', inplace=True):
        
        self.fit(data, y, features)
        if inplace:
            for col in self.values:
                for item, value in self.values[col].iteritems():
                    data.ix[data[col] == item, col] = value
        else:
            new_data = data.copy()
            for col in self.values:
                for item, value in self.values[col].iteritems():
                    new_data.ix[data[col] == item, col] = value
                    
            return new_data
    
    def transform(self, data, inplace=True):
        import warnings

        if inplace:
            for col in self.values:
                if col in data.columns:
                    for item, value in self.values[col].iteritems():
                        data.ix[data[col] == item, col] = value
                else:
                    warnings.warn('Column ' + col + ' is missed in this dataset.')
        else:
            new_data = data.copy()
            for col in self.values:
                if col in new_data.columns:
                    for item, value in self.values[col].iteritems():
                        new_data.ix[data[col] == item, col] = value
                else:
                    warnings.warn('Column ' + col + ' is missed in this dataset.')
            return new_data
    

In [35]:
train, test, y = reload_data()
se = SemenovEncoding(C=10)
a = se.fit_transform(train, y, features=['Gender'], inplace=False)

In [36]:
a.Gender.unique()

array([0.3078560935049717, 0.15500003356966377, 0.04634204747219886], dtype=object)